In [7]:
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
%load_ext autoreload
%autoreload 2
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# O método da posição falsa

## $ \S 1 $ Descrição do método da posição falsa

Suponha que a função real contínua $ f \colon [a, b] \to \mathbb R $ seja tal que
$$
    \operatorname{sinal} f(a) \ne \operatorname{sinal} f(b)\,.
$$
Assim como o método da bissecção, o __método da posição falsa__ começa com as
duas estimativas $ a $ e $ b $ para um zero. Mas em vez de tomar a próxima
estimativa como o ponto médio destes, construímos a reta pelos pontos
$$
\big(a, f(a)\big) \quad \text{e} \quad \big(b,f(b)\big)
$$
e encontramos o ponto onde ela cruza o eixo-$x$, definindo $ c $ como sua
primeira coordenada. Este seria exatamente o zero de $ f $ caso ela fosse
linear entre $ a $ e $ b $, mas em geral isto não acontecerá. Portanto há três
possibilidades:
* Se $ f(c) = 0 $, então $ c $ é um zero de $ f $ e podemos terminar.
* Se $ \operatorname{sinal} f(c) \neq \operatorname{sinal} f(a) $, então $ f $ troca de sinal em $ [a, c] $. Neste caso fazemos $ b \leftarrow c $.
* Se $ \operatorname{sinal} f(c) = \operatorname{sinal} f(a) $, então $ f $ troca de sinal em $ [c, b] $. Neste caso fazemos $ a \leftarrow c $.

Nos dois últimos casos repetimos o procedimento usando o novo intervalo $ [a, b] $ em lugar do original e assim sucessivamente, até que seja satisfeito o critério de parada. Observe que em cada iteração temos a garantia da existência de um zero dentro do intervalo sob consideração, pois $ f $ troca de sinal aí.

📝 O método da posição falsa também é conhecido como _regula falsi_.

__Exemplo 1:__ Execute em seqüência as três células abaixo para ver uma animação
do método da posição falsa aproximando o zero da função $ f(x) = 1 + x\cos x + \sin(2x) $
em $ 1.8394 $ (aproximadamente).

In [2]:
def print_solution(xs: list[float], ys: list[float], freq: int = 1) -> None:
    """
    Given two arrays xs and ys of the same length, prints a table whose n-th
    line consists of three entries: the values of n, xs[n] and ys[n].
    Parameters:
        * The arrays xs and ys. 
        * A parameter freq used to print only one in every freq line. The
          first, second and last line are always printed. If freq == 0, then
          only these lines are printed.
    Output: None.
    Prints: A header and the table described above.
    """
    def print_header() -> None:
        """
        Prints the table's header.
        """
        print("\n|       n      ", end="")
        print("    x_n            ", end="")
        print("    f(x_n)      |")
        print("|=================================================|")
        
    def print_line(n: int, x: float, y: float) -> None:
        """
        Pretty-prints n, x and y.
        """
        if n == 0:
            print(f"|       a", end="")
        elif n == 1:
            print(f"|       b", end="")
        else:
            print(f"|      {n - 1:02}", end="")
        print(f"    {x:15.8f}", end="")
        print(f"    {y:15.8f}   |")
    
    
    N = len(xs)
    if freq == 0:       # If freq == 0, print only first and last lines.
        freq = N - 1
    print_header()
    for n in range(0, 2):
        print_line(n, xs[n], ys[n])
    for n in range(2, N, freq):
        print_line(n, xs[n], ys[n])
    if n != N - 1:
        print_line(n, xs[N], ys[N])
    print("|_________________________________________________|\n")
        
    return None

In [6]:

def false_position_animation(f: Callable[[float], float], a: float, b :float,
                        N: int = 4, title: str = "", duration: float = 0.75
                        ) -> tuple[list[float], list[float]]:
    """
    Displays an animation of the false position (a.k.a. regula falsi) method
    applied to a function.
    Parameters:
        * A continuous real function f.
        * The two endpoitns a and b of an interval such that f(a)f(b) < 0.
        * The maximum number N of iterations.
        * A title to be displayed at the top of the diagram.
        * The pause between slides of the animation, in seconds.
          Set pause = 0 to produce a figure instead of an animation.
    Output:
        * Two lists xs and ys containing the estimates and the values of the
          function f at each of them.
    Displays:
        * The animation in a pop-up window.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    %matplotlib qt
    
    
    def pause(duration):
        """
        Pauses the animation for duration seconds, provided duration > 0.
        """
        if duration > 0:
            plt.pause(duration)

    def iterate(a, b):
        """
        Applies a single step of the false position method to the interval
        [a, b]. Returns the next estimate and the left, right endpoints
        of the next interval.
        """
        c = (a * f(b) - b * f(a)) / (f(b) - f(a)) 
        if np.sign(f(a)) != np.sign(f(c)):     # [a, c] contains a zero.
            return c, a, c
        else:                                  # [c, b] contains a zero.
            return c, c, b
    
    
    P = 200                                    # Number of points in each plot.
    width = 1.75                               # Line width.
    marker_size = 5
    domain = np.linspace(a, b, P)              # Generates P nodes from a to b.
    xs = [a, b]                                # Stores the estimates.
    ys = [f(a), f(b)]                          # Stores f of the estimates.
    # Creating lists to store the sample points for the lines between
    # consecutive estimates:
    xs_line = [np.linspace(a, b, P)]
    ys_line = [np.linspace(f(a), f(b), P)]
    for _ in range(N):                         # Filling xs.
        c, a, b = iterate(a, b)
        xs.append(c)
        xs_line.append(np.linspace(a, b, P))   # Filling xs_line.
        ys_line.append(np.linspace(f(a), f(b), P))    # Filling ys_line.
    ys = [f(x) for x in xs]                    # Stores f of the estimates.
    # Lists containing the x and y coordinates for plotting vertical lines:
    xs_vert = [np.linspace(xs[n], xs[n], P) for n in range(N + 2)]
    ys_vert = [np.linspace(0, ys[n], P) for n in range(N + 2)]
    cmap = plt.get_cmap("tab10")               # Used to control the colors.
    
    # Draw the graph of f:
    plt.axhline(y=0.0, color='black', linestyle='-', lw=width)
    plt.xlabel("$ x $-axis")
    plt.ylabel("$ y $-axis")
    plt.title(title)
    plt.grid(True)
    plt.plot(domain, f(domain), label="$ y = f(x) $", lw=width)
    plt.legend()

    # Mark a on the x-axis and draw the vertical line x = a:
    pause(duration)
    plt.plot(xs[0], 0, color='black', marker="|", mew=width)
    pause(duration)
    plt.plot(xs_vert[0], ys_vert[0], linestyle='-', lw=width, label='$ a $')
    plt.plot(xs[0], 0, color='black', marker="|", mew=width)
    plt.plot(xs[0], ys[0], color='black', marker="o", ms=marker_size)
    plt.legend()
    
    # Mark b on the x-axis and draw the vertical line x = b:
    pause(duration)
    plt.plot(xs[1], 0, color='black', marker="|", mew=width)
    pause(duration)
    plt.plot(xs_vert[1], ys_vert[1], linestyle='-', lw=width, label='$ b $')
    plt.plot(xs[1], 0, color='black', marker="|", mew=width)
    plt.plot(xs[1], ys[1], color='black', marker="o", ms=marker_size)
    plt.legend()

    for n in range(2, N + 2):
        pause(duration)
        # Drawing the line between the two previous estimates:
        plt.plot(xs_line[n - 2], ys_line[n - 2],
                 linestyle='--', lw=width)
        pause(duration)
        # Marking x_n:
        plt.plot(xs[n], 0, color='black', marker="x", mew=width)
        pause(duration)
        # Drawing the vertical line through x_n and marking (x_n, y_n):
        plt.plot(xs_vert[n], ys_vert[n], linestyle='-', lw=width,
                 label=f'$ x_{n - 1} $', color=cmap(n + 1))
        plt.plot(xs[n], 0, color='black', marker="x", mew=width)
        plt.plot(xs[n], ys[n], color='black', marker="o", ms=marker_size)
    plt.legend()
    
    return xs, ys

In [5]:
from numpy import sin, cos, pi
a = 0           # Extremidade esquerda do intervalo inicial, onde f vale -3.
b = pi          # Extremidade direita, onde f vale 3.
N = 4           # Número de iterações desejado.
pausa = 0.0     # Intervalo de tempo entre cada passo da animação, em segundos.
# Função à qual o método será aplicado:
f = lambda x: 1 + x * cos(x) + sin(2 * x)
# Título a ser exibido no topo do diagrama:
titulo = "Método da posição falsa para "\
         "$ y = 1 + x\cos x + \sin(2x),\ a = 0,\ b = \pi $."

xs, ys = false_position_animation(f, a, b, N, titulo, pausa)

![Exemplo do método da posição falsa](fig_2-4_exemplo_1.png "Exemplo de aplicação do método da posição falsa")

In [44]:
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|       a         0.00000000         1.00000000   |
|       b         3.14159265        -2.14159265   |
|      01         1.00000000         2.44959973   |
|      02         2.14263232        -1.06970379   |
|      03         1.79532549         0.16615807   |
|      04         1.84201990        -0.00972982   |
|_________________________________________________|



## $ \S 2 $ Fórmulas para a próxima estimativa e para o erro

### $ 2.1 $ Fórmula para a próxima estimativa

A reta por dois pontos $ (x_0, y_0) $ e $ (x_1, y_1) $ é caracterizada pela igualdade da inclinação do segmento que liga $ (x_1, y_1) $ a $ (x_0, y_0) $ e a do segmento ligando um ponto $ (x, y) $ qualquer sobre a reta a $ (x_0, y_0) $. Em símbolos:
$$
\frac{y - y_0}{x - x_0} = \frac{y_1 - y_0}{x_1 - x_0}
$$
Equivalentemente, esta reta é descrita pela equação
$$
y = y_0 + \frac{y_1 - y_0}{x_1 - x_0}\,(x - x_0).
$$

No método da posição falsa utilizamos em cada passo a reta passando por $ \big(a,f(a)\big) $ e $ \big(b,f(b)\big) $, onde $ a $ e $ b $ são as extremidades do intervalo obtido no passo anterior. Substituindo estes valores acima, obtemos a equação
$$
y = f(a) + \frac{f(b) - f(a)}{b - a}\,(x - a).
$$
A próxima estimativa $ \xi $ para o zero é o único valor de $ x $ que faz esta expressão se anular:
$$
\boxed{\xi = \frac{af(b) - bf(a)}{f(b)-f(a)}}
$$ 

📝 Observe a simetria desta fórmula com respeito a $ a $ e $ b $ e o fato que o denominador é não-nulo pois $ f(a) $ e $ f(b) $ têm sinais opostos por hipótese.

### $ 2.2 $ Análise informal do erro

Suponha por concretude que o sinal de $ f(\xi) $ seja o mesmo que o de $ f(a) $, de modo que o intervalo seguinte seja $ [\xi, b] $. Podemos estimar o novo erro $ b - \xi $ em termos do anterior $ b - a $:
\begin{alignat*}{3}
b - \xi &= \frac{bf(b) - {bf(a)} -\big[af(b) - {bf(a)}\big]}{f(b) - f(a)} \\
& = \frac{f(b)}{f(b) - f(a)} (b - a)\,.
\end{alignat*}
Como por hipótese $ f(a) $ e $ f(b) $ têm sinais opostos, o fator que multiplica $ (b - a) $ está entre $ 0 $ e $ 1 $. Se $ f(b) $ for muito maior que $ f(a) $ em valor absoluto, este fator ficará próximo de $ 1 $. Se isto acontecer em todas as iterações, a convergência ao zero será lenta; esta situação é ilustrada pela animação abaixo.

In [10]:
a = 0          # Extremidade esquerda do intervalo inicial, onde f vale -3.
b = 4          # Extremidade direita, onde f vale 57.
N = 8          # Número de iterações desejado.
pausa = 0.5    # Intervalo de tempo entre cada passo da animação, em segundos.
f = lambda x: x**4 - x - 3    # Função à qual o método será aplicado.
# Título a ser exibido no topo do diagrama:
titulo = ("Método da posição falsa para $ y = x^3 - x - 3,\ a = 0,\ b = 4 $."
          "\nAproximação lenta por um único lado.")

xs, ys = false_position_animation(f, a, b, N, titulo, pausa)
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|       a         0.00000000        -3.00000000   |
|       b         4.00000000       249.00000000   |
|      01         0.04761905        -3.04761391   |
|      02         0.09540895        -3.09532609   |
|      03         0.14335106        -3.14292878   |
|      04         0.19142369        -3.19008098   |
|      05         0.23960032        -3.23630460   |
|      06         0.28784793        -3.28098274   |
|      07         0.33612547        -3.32336092   |
|      08         0.38438251        -3.36255247   |
|_________________________________________________|



![Exemplo de convergência lenta com o método da posição falsa](fig_2-4_exemplo_2.png "Exemplo de convergência lenta com o método da posição falsa")

Não é possível calcular precisamente e de maneira geral o erro cometido pelo método da posição falsa. Porém na situação descrita acima, seu desempenho é consideravelmente pior que o do método do bissecção. Por este motivo ele raramente é empregado na prática.

📝 Para exemplos ainda mais extremos da lentidão da convergência, troque o expoente $ n = 3 $ por outro maior na expressão para a função $ f(x) = x^n - x - 3 $ considerada acima e rode novamente a animação.

## $ \S 3 $ Implementação do método da posição falsa

Como o método da bissecção e da posição falsa só diferem na fórmula usada para obter a nova estimativa para o zero, as duas implementações são muito parecidas.

In [12]:
def false_position(f: Callable[[float], float], a: float, b: float,
                   eps: float = 1e-3, max_iter: int = 100
                   )-> tuple[list[float], list[float]]:
    """
    Uses the false position (regular falsi) method to approximate a zero of a
    function.
    Parameters:
        * A real continuous function f.
        * Points a and b such that f(a)f(b) < 0 and f is defined on [a, b].
        * The maximum tolerance eps for the error.
        * The maximum number max_iter of iterations.
    Output:
        * Two lists, xs and ys, containing the estimates and the values of f
          at each of them, respectively.
    Prints:
        * The last estimate.
        * The value of f at this estimate.
        * The number of iterations that were performed.
        * An upper bound for the error.
    """
    from numpy import sign
    

    iterations = 0                        # Counts the number of iterations.
    f_a = f(a)                            # Storing the value of f at a.
    f_b = f(b)                            # Storing the value of f at b.
    xs = [a, b]                           # List to store the estimates.
    ys = [f_a, f_b]                       # List to store f of the estimates.
    if eps <= 0:                          # Error: invalid value for 'eps'.
        raise ValueError("The tolerance must be positive!")
    if f_a == 0:                          # a is a zero.
        print("a is an exact zero.")
        return a
    elif f_b == 0:                        # b is a zero.
        print("b is an exact zero.")
        return b
    elif sign(f_a) == sign(f_b):          # Error: cannot guarantee zero exists.
        raise ValueError("The function takes on the same sign"
                         "at the given endpoints!")
    
    while (b - a) >= eps and iterations <= max_iter:
        c = (a * f(b) - b * f(a)) / (f(b) - f(a))       # Next estimate.
        f_c = f(c)                        # Storing the value of f at c.
        xs.append(c)                      # Appending c to the estimates.
        ys.append(f_c)                    # Appending f(c) to ys.
        if f_m == 0:
            print("Found an exact zero.")
            return c
        elif sign(f_a) != sign(f_c):      # [a, c] contains a zero.
            b = c                         # Take c to be the new b.
        else:                             # [c, b] contains a zero.
            a = c                         # Take c to be the new a.
            f_a = f_c
        iterations += 1
        
    print(f"Found an approximate zero:\n{m:15.9f}")
    print(f"after {iterations} iterations, with an error of at most {b - a}.")
    print(f"The value of f at this point is:\n{f(m):15.9f}")
    
    return xs, ys

## $ \S 4 $ Problemas

**Problema 1:** Usando o método da posição falsa com precisão de três dígitos decimais:

(a) Encontre uma raiz positiva da equação $ \sin x = \frac{x}{2} $.

(a) Encontre a primeira raiz positiva da equação $ \sin x = \frac{x}{n} $ para $ n = 2, 3, \dots, 20 $.

*Solução:*

**Problema 2:** Calcule com ajuda do computador, mas sem usar a implementação acima, as três primeiras iterações do método da posição falsa para estimar uma raiz das equações abaixo nos intervalos indicados. Esboce também os gráficos das funções utilizadas:

(a) $ \tan x - \frac{1}{1 + x^2} = 0 $, $ 0 \le x \le \frac{\pi}{2} $.

(b) $ x^2 = 2 $, $ 0 \le x \le 2 $.

(c) $ x \ln x = 1 $, $ 1 \le x \le 2 $.

(d) $ \cos x = x $, $ 0 \le x \le \frac{\pi}{2} $.

*Solução:*

**Problema 3:** Vimos no caderno anterior que no método da bissecção os comprimentos do intervalo anterior e do atual estão relacionados pela fórmula 
$$
\left\vert{I_n}\right\vert = c \left\vert{I_{n-1}}\right\vert \quad \text{com} \quad c = \frac{1}{2}
$$ 

(a) Estime a constante $ c $ que relaciona as duas para o método da posição falsa aplicado ao problema de se encontrar a raiz positiva da equação
$$
x^{8} - 1 = 0
$$
usando como intervalo inicial $ [a, b] = [0, 2] $.

(b) Conclua se seria melhor usar o método da bissecção ou o método da posição falsa neste caso. 